### collocation

In [1]:
# REF https://m.blog.naver.com/vangarang/221076777192

In [2]:
from konlpy.corpus import kolaw
print( len( kolaw.open('constitution.txt').read() ) )

18884


In [3]:
from nltk.collocations import BigramAssocMeasures
from nltk.collocations import TrigramAssocMeasures
from nltk.metrics.association import QuadgramAssocMeasures
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import TrigramCollocationFinder
from nltk.collocations import QuadgramCollocationFinder

ngram = [(BigramAssocMeasures(),BigramCollocationFinder),
         (TrigramAssocMeasures(),TrigramCollocationFinder),
         (QuadgramAssocMeasures(),QuadgramCollocationFinder)]

In [4]:
from konlpy.tag import Okt
s = kolaw.open('constitution.txt').read() # 한국 법률 말뭉치
okt = Okt()      # loading tagger
tokens = okt.pos(s, norm=True, stem=True)  # pos tagging

In [5]:
'''각각의 AssocMeasures들은 단어 간의 연관도 측정 알고리즘을 담고있습니다. 
연관도 측정 방법은 연관도를 정의하는 방법에 따라 다양하게 나눠지는데요. 
NLTK에서는 이 알고리즘들을 함수로 각각 구현해놓았습니다. 
각각 pmi, raw_freq, student_t, chi_sq, mi_like, poisson_stirling, jaccard, likelihood_ratio (8) 입니다.'''

'각각의 AssocMeasures들은 단어 간의 연관도 측정 알고리즘을 담고있습니다. \n연관도 측정 방법은 연관도를 정의하는 방법에 따라 다양하게 나눠지는데요. \nNLTK에서는 이 알고리즘들을 함수로 각각 구현해놓았습니다. \n각각 pmi, raw_freq, student_t, chi_sq, mi_like, poisson_stirling, jaccard, likelihood_ratio (8) 입니다.'

In [6]:
founds_from_4measure = []

for measure, finder in ngram:
    finder = finder.from_words(tokens)
    founds = finder.nbest(measure.pmi, 10)       # pmi - 상위 10개 추출
    founds += finder.nbest(measure.chi_sq, 10)   # chi_sq - 상위 10개 추출
    founds += finder.nbest(measure.mi_like, 10)  # mi_like - 상위 10개 추출
    founds += finder.nbest(measure.jaccard, 10)  # jaccard - 상위 10개 추출

    founds_from_4measure += founds

In [7]:
from konlpy.utils import pprint
from collections import Counter
collocations = [' '.join([w for w,t in collocation]) for collocation in founds_from_4measure]
collocations = [(w,f) for w,f in Counter(collocations).most_common() if f > 2]
pprint(collocations)

[('1948년 7월 12일', 4),
 ('1988년 2월 25일', 4),
 ('2월 25일 부터', 4),
 ('7월 12일 에', 4),
 ('가부 동 수인', 4),
 ('국립 대학교 총장', 4),
 ('군 경의 유가족', 4),
 ('농수산 물의 수급', 4),
 ('선거일 현재 40', 4),
 ('우호 통상 항해', 4),
 ('1948년 7월 12일 에', 4),
 ('1988년 2월 25일 부터', 4),
 ('전몰 군 경의 유가족', 4),
 ('\n\n  펼치다 부칙 <', 4),
 ('29 .> 부칙 보기', 4),
 ('70일 내지 40일 전에', 4),
 ('계 도하 고 생산품', 4),
 ('빛나다 우리 대 한', 4),
 ('선거일 현재 40 세', 4),
 ('.> 부칙 보기 \n\n', 4),
 ('\n\n  펼치다', 3),
 ('" 나', 3),
 ('12일 에', 3),
 ('1948년 7월', 3),
 ('1988년 2월', 3),
 ('25일 부터', 3),
 ('29 .>', 3),
 ('2월 25일', 3),
 ('40조 입법권', 3),
 ('58조 국채', 3)]


### chunk

In [ ]:
# REF https://m.blog.naver.com/vangarang/221076896346

In [8]:
import konlpy
import nltk

# POS tag a sentence
sentence = u'만 6세 이하의 초등학교 취학 전 자녀를 양육하기 위해서는'
words = okt.pos(sentence)

# Define a chunk grammar, or chunking rules, then chunk
grammar = """
NP: {<N.*>*<Suffix>?}   # Noun phrase
"""
parser = nltk.RegexpParser(grammar)
chunks = parser.parse(words)
print("# Print whole tree")
chunks.pprint()

# Print whole tree
(S
  (NP 만/Noun 6/Number 세/Noun 이하/Noun)
  의/Josa
  (NP 초등학교/Noun 취학/Noun 전/Noun 자녀/Noun)
  를/Josa
  (NP 양육/Noun)
  하기/Verb
  (NP 위/Noun)
  해서는/Verb)


In [9]:
chunks = [ chunk.leaves() for chunk in chunks if type(chunk) != tuple ]

In [11]:
chunks

[[('만', 'Noun'), ('6', 'Number'), ('세', 'Noun'), ('이하', 'Noun')],
 [('초등학교', 'Noun'), ('취학', 'Noun'), ('전', 'Noun'), ('자녀', 'Noun')],
 [('양육', 'Noun')],
 [('위', 'Noun')]]

In [12]:
import re

# chunk를 본문과 같이 만들고 빈도 수 카운팅
result = set()
for chunk in chunks:
    seq = [w for w, t in chunk]
    pattern = '[ ]?'.join(seq)
    if len(pattern) < 2:
        continue
    seq_pattern = re.compile(pattern)
    founds = re.findall(seq_pattern, sentence)
    len_founds = len(founds)
    if not len_founds:
        continue
    ele = (founds[0], len_founds)
    if len(set(founds)) > 1:
        ele = (tuple(set(founds)), len_founds)
    result.add(ele)

In [13]:
result

{('만 6세 이하', 1), ('양육', 1), ('초등학교 취학 전 자녀', 1)}

In [14]:
import re

In [15]:
# POS tag a sentence
s = kolaw.open('constitution.txt').read() # 한국 법률 말뭉치
words = okt.pos(s)

# Define a chunk grammar, or chunking rules, then chunk # Noun phrase
grammar = 'NP: {<N.*>*<Suffix>?}'
parser = nltk.RegexpParser(grammar)
chunks = parser.parse(words)
chunks = [ chunk.leaves() for chunk in chunks if type(chunk) != tuple ]

# chunk를 본문과 같이 만들고 빈도 수 카운팅
result = set()
for chunk in chunks:
    seq = [w for w, t in chunk]
    pattern = '[ ]?'.join(seq)
    if len(pattern) < 2:
        continue
    seq_pattern = re.compile(pattern)
    founds = re.findall(seq_pattern, s)
    len_founds = len(founds)
    if not len_founds:
        continue
    ele = (founds[0], len_founds)
    if len(set(founds)) > 1:
        ele = (tuple(set(founds)), len_founds)
    result.add(ele)

result = sorted(list(result), key=lambda x:x[1], reverse=True)
pprint(result[80:100]) # print chunks and counting

[('구성', 9),
 ('정치', 9),
 ('자치', 9),
 ('법률안', 9),
 ('사회', 9),
 ('행위', 9),
 ('체포', 9),
 ('효력', 9),
 ('투표', 9),
 ('대하여', 9),
 ('대하', 9),
 ('행정각부', 9),
 ('선거관리', 9),
 ('생활', 9),
 ('기본', 9),
 ('명령', 9),
 ('통일', 9),
 ('절차', 9),
 ('국정', 9),
 ('제정', 8)]
